<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2022/ex02noteA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex02noteA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


----
## 回帰のための教師あり学習(1) 直線の当てはめ
----

----
### 準備

Google Colab の Notebook では， Python というプログラミング言語のコードを動かして計算したりグラフを描いたりできます．
Python は，機械学習・人工知能やデータサイエンスの分野ではメジャーなプログラミング言語ですが，それを学ぶことはこの授業の守備範囲ではありません．以下の所々に現れるプログラムっぽい記述の内容は，理解できなくて構いません．

以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# データを読み込む
dfGori = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/gorigori.csv', header=0)
xmin, xmax = -5, 40
ymin, ymax = 0, 130

----
### 復習 ― ゴリゴリ君ふたたび

前回の「ゴリゴリ君」の例は，「気温」と「アイス売上数」の数値を集めた以下のようなデータが与えられたときに，気温からアイス売上数を予測する，という問題でした（次のセルを実行してね）．


In [ ]:
dfGori

気温が $x$ [度] のときのアイス売上数の予測値を $y$ [個] として，$x$ から $y$ を予測する仕組みとして

$$
\begin{aligned}
y = f(x) = ax+b
\end{aligned}
$$

という式を考えて，予測が「うまくいく」ようにパラメータ $(a, b)$ を決めたい，という話になっていました．

In [ ]:
#@title a, b の値を適当に変えてこのセルを何度か実行し直してみよう
a = 1.5#@param　{type:"number"}
b = 35 #@param　{type:"number"}

X = dfGori['気温']
Y = dfGori['アイス売上数']

Xr = np.array([xmin, xmax])
Yest = a * Xr + b

fig, ax = plt.subplots(1, facecolor='white', figsize=(6, 4))
ax.scatter(X, Y)
ax.plot(Xr, Yest, color='red')
ax.set_xlim(xmin, xmax)
ax.set_ylim(0, 130)
plt.show()

for x in [10, 20, 30]:
    y = a*x+b
    print(f'気温 {x} 度のときのアイス売上数の予測値は {y:.1f} 個')

前回は手動でパラメータを変化させてみて，なんとなくよさそうな値を選んでいましたが，そんないい加減な方法は使い物になりません．
このような問題をコンピュータに解かせるためには，問題をきちんと定式化（数式で表現）し，その解を計算によって求められるようにしなければなりません．

Q. この問題は，教師あり学習／教師なし学習のどちらですか．また，回帰／識別のどちらですか．

----
### 直線当てはめの問題設定

「ゴリゴリ君」の例は，2次元の平面上に $(気温, アイス売上数)$ という点がたくさん散らばっているときに，それらの点たちに当てはまる直線の式（を表す2つのパラメータ）を求める問題となっています．


この問題は，一般化すると次のように表せます：

> $x$ の値から $y$ の値が決まるようなデータがあり，両者の関係として直線の式 $ y = ax+b $ が仮定できるとする．$x$ と $y$ の値のペアが $(x_1, y_1), (x_2, y_2), \ldots, (x_N, y_N)$ と $N$ 個与えられたときに，$x_n$ を式に代入して得られる値 $ax_n + b$ が $y_n$ の「よい」近似値となるような，つまり
> 
> $$
y_n \approx ax_n + b \qquad (n = 1, 2, \ldots , N)
> $$
> 
> となるようなパラメータ $(a, b)$ を求めたい（記号 $\approx$ は「近似的に等しい」という意味）．




ただ，これだとまだ「よい」とはどういうことかがあいまいです．ここをきちんとするために，当てはまりのよさの規準を定めましょう．
そのために，入力 $x_n$ を式に代入して得られる出力 $ax_n+b$ と，その正解の値 $y_n$ との間の差の二乗である **二乗誤差** （squared error）

$$
(y_n - (ax_n+b))^2 \qquad (n = 1, 2, \ldots , N)
$$

を考えます（下図参照）．

<img width="50%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/residual.png">

このとき，$N$ 個のデータについてのこの二乗誤差の和
$$
\sum_{n=1}^N (y_n - (ax_n+b))^2
$$
は，$0$ 以上の実数値をとり，$(a, b)$ を変えるとその値が変化します．この値が小さければ小さいほど「データへの直線の当てはまりがよい」と考えることができそうです．
以降では，この考え方に基づいて話を進めます（注）．

※注: ここでは二乗誤差を規準としていますが，「差の絶対値」など別の規準を用いることも可能です．その辺りのことはこの授業では説明しません．興味があれば takataka に尋ねてみてください．


----
### 最小二乗法 ― 二乗誤差を最小にするパラメータを求める


上述のように，当てはまりのよさの規準として出力の値と正解の値との間の二乗誤差の和を考え，その値が最小となるようなパラメータを求める方法を，**最小二乗法** （least squares method）といいます（最小となるパラメータをどうやって求めるかは後述）．機械学習よりも歴史の長い，理工学の幅広い分野でよく使われるデータ分析手法ですが，機械学習の文脈では，回帰のための最も基本的な手法と位置づけることができます．

ここまでで問題をきちんと定式化できましたので，あとは数学の問題となります．やりたいことは，データ $(x_1, y_1), \ldots, (x_N, y_N)$ に対して二乗誤差の和を最小にするパラメータ $(a, b)$ を求めることです．以下にその解法を説明します．

関数 $E(a, b)$ を次式で定めます．

$$ E(a, b) = \frac{1}{2}\sum_{n=1}^N (y_n - (ax_n+b))^2 $$

先に出てきた二乗誤差の和を $\frac{1}{2}$ 倍しただけのものですので，これを最小にする $(a, b)$ を求めてもよいのは明らかですね．（注）．

<span style="font-size: 75%">
※注: そうするのは，微分した後がほんのちょびっと楽だからってだけです．
</span>


この値が最小となる $(a, b)$ を求めるため，$\frac{\partial E(a,b)}{\partial a} = 0$，$\frac{\partial E(a,b)}{\partial b} = 0$ とおきます（注）．

<span style="font-size: 75%">
※注: $E(a, b)$ は「下に凸な」関数なので，その偏導関数がいずれも $0$ のときに最小となります．
</span>

すると，次式が得られます．

$$
\begin{aligned}
\frac{\partial E(a,b)}{\partial a} &= \sum_{n=1}^{N}(y_n-(ax_n+b))(-x_n) = a \sum_{n=1}^{N}x_n^2 + b\sum_{n=1}^{N}x_n - \sum_{n=1}^{N}x_ny_n = 0\\
\frac{\partial E(a,b)}{\partial b} &= \sum_{n=1}^{N}(y_n-(ax_n+b))(-1) = a \sum_{n=1}^{N}x_n + b\sum_{n=1}^{N}1 - \sum_{n=1}^{N}y_n = 0\\
\end{aligned}
$$


これを整理すると，次の連立一次方程式が得られます．

$$
\left\{
\begin{aligned}
a \sum_{n=1}^{N}x_n^2 + b\sum_{n=1}^{N}x_n &= \sum_{n=1}^{N}x_ny_n\\
a \sum_{n=1}^{N}x_n + b\sum_{n=1}^{N}1 &= \sum_{n=1}^{N}y_n
\end{aligned}
\right.
$$

行列の式として書くとこんなん：
$$
\begin{pmatrix}
\sum_{n=1}^Nx_n^2 & \sum_{n=1}^Nx_n \\ 
\sum_{n=1}^Nx_n & \sum_{n=1}^N1
\end{pmatrix}
\begin{pmatrix}
a \\ b
\end{pmatrix}
=
\begin{pmatrix}
\sum_{n=1}^N x_ny_n \\ \sum_{n=1}^N y_n
\end{pmatrix}
$$

この連立方程式を「正規方程式」といいます．
二乗誤差の和を最小にするパラメータ $(a, b)$ は，この連立方程式の解として求まります．

----
### ゴリゴリ君で最小二乗法やってみる

「ゴリゴリ君」のデータで実際にコンピュータに直線当てはめの最小二乗法を解かせてみましょう．

In [ ]:
# データを用意
X = dfGori['気温']
XX = np.vstack([X, np.ones_like(X)]).T
Y = dfGori['アイス売上数']

# 最小二乗法の解を求める
XTX = XX.T @ XX  # 正規方程式左辺の 2x2 行列
XTY = XX.T @ Y   # 正規方程式右辺の ベクトル
a, b = np.linalg.solve(XTX, XTY)

# 直線の式を計算
Xr = np.array([xmin, xmax])
Yest = a * Xr + b

# グラフを描く
fig, ax = plt.subplots(1, facecolor='white', figsize=(6, 4))
ax.scatter(X, Y)
ax.plot(Xr, Yest, color='red')
ax.set_xlim(xmin, xmax)
ax.set_ylim(0, 130)
plt.show()

# 予測値を求める
print(f'得られたパラメータ (a, b) = ({a:.3f}, {b:.3f})')
for x in [10, 20, 30]:
    y = a*x+b
    print(f'気温 {x} 度のときのアイス売上数の予測値は {y:.1f} 個')

----
### 練習問題

手計算で最小二乗法やってみよう．

［問題］

ほげおくんは，こつこつためたお金で自動車を購入したいと重い，たまに中古車情報を調べている．ほげおくんが気になっている自動車の価格は，調べ始めた日には30万円だったが，それ以降，表のような推移を示していた．

|$n$|経過週数 $x_n$|価格[万円]|価格差 $y_n$ [万円]|
|----:|----:|----:|----:|
|1|0| 30 | 0|
|2|4|31|1|
|3|8|29|-1|
|4|20|28|-2|

$(x_1, y_1)$ から $(x_4, y_4)$ までの $N=4$ 個のデータについて，経過週数を$x$，価格**差**を$y$として，
$$
y = ax+b
$$
という直線を当てはめたい．最小二乗法を用いてこの直線の式を求め，ほげおくんが気になっている自動車の価格推移を予測してみよう．


(1) 上記のデータについて，次の値をそれぞれ求めなさい．

$$
\sum_{n=1}^{N}x_n,\quad\sum_{n=1}^{N}y_n,\quad\sum_{n=1}^{N}x_n^2,\quad\sum_{n=1}^{N}x_ny_n
$$

(2) 正規方程式を解いて解 $(𝑎,𝑏)$ を求めなさい．

(3) (2)の結果を利用して，経過週数から価格差を予測する式を求めなさい．それを用いて，ほげおくんが気になっている自動車の価格の，調べ始めから36週後の価格はいくらと予想されるか答えなさい．